In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

sys.path.append('../src')
from dota import Match
from youtube import (
    analyze_video, crop_frame, extract_text, convert_to_timestamp, get_video_metadata,
    get_nearest_matches, generate_team_pairs, search_team_pairs
)
from utils import plot_player_signals, TimeTable, calculate_iou
from settings import FRAMES_DIR, YOUTUBE_DIR

# Quick Start

In [ ]:
match = Match.from_id(6676393091)
match

In [ ]:
match.parse()

In [ ]:
match.get_action_moments()

# Details
## Highlights Detector

In [ ]:
player = match.players[1]
player

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15, 5))
plot_player_signals(
    player,
    ax=ax[0],
    hp=True,
)

plot_player_signals(
    player,
    ax=ax[1],
    dhp=True
)

plot_player_signals(
    player,
    ax=ax[2],
    sdhp=True
)

zoom_start, zoom_end = 600, 800
fig, ax = plt.subplots(figsize=(15, 5))
plot_player_signals(
    player,
    ax=ax,
    zoom_start=zoom_start,
    zoom_end=zoom_end,
    hp=True,
    sdhp=True,
)

plt.grid(True)
plt.show()

In [ ]:
zoom_start, zoom_end = 600, 800
fig, ax = plt.subplots(figsize=(15, 5))
plot_player_signals(
    player,
    ax=ax,
    zoom_start=zoom_start,
    zoom_end=zoom_end,
    # use_minutes=True,
    hp=True,
    # max_hp=True,
    # dhp=True,
    sdhp=True,
    signal_hp_decreasing=True,
    signal_hp_low=True,
    # deaths=True,
    # as_target=True,
    as_target_escaped=True,
    # as_attacker=True,
    # as_attacker_kill=True,
    action_moments=True,
)

plt.grid(True)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
plot_player_signals(
    player,
    ax=ax,
    use_minutes=True,
    hp=True,
    # max_hp=True,
    # dhp=True,
    # sdhp=True,
    signal_hp_decreasing=True,
    signal_hp_low=True,
    # deaths=True,
    as_target=True,
    as_target_line_level=-150,
    # as_target_escaped=True,
    as_attacker=True,
    as_attacker_line_level=-300,
    # as_attacker_kill=True,
    action_moments=True,
    action_moments_line_level=-450,
)

plt.grid(True)
plt.show()

Problems:
- The laning phase is usually boring
- What moments are interesting?

Hueristics:
- 1x1
- Mass Fights
- Skip Laning Phase
- Calculate Damage
- Check for epic abilities: Black Hole, Reverse Polarity
...

## YouTube

In [ ]:
url = 'https://www.youtube.com/watch?v=XTngObUROMo&ab_channel=DotADigest'

In [ ]:
analyze_video(
    url=url,
    keep_video=True,
    keep_frames=True,
    force_process=True,
    frames_limit=30,
    min_confidence=0,
)

In [ ]:
os.listdir(FRAMES_DIR)[:10]

In [ ]:
frame_path = FRAMES_DIR / 'XTngObUROMo__539.bmp'
Image.open(frame_path)

In [ ]:
crop = crop_frame(frame_path)
crop[:1]

In [ ]:
Image.fromarray(crop)

In [ ]:
frame_text = extract_text(crop)
frame_text

In [ ]:
convert_to_timestamp(frame_text)

In [ ]:
video_id, metadata = get_video_metadata(url)
title = metadata['fulltitle']
upload_date = datetime.strptime(metadata['upload_date'], '%Y%m%d')
print(title, '\n')
print(upload_date)

In [ ]:
matches = get_nearest_matches(upload_date)
print(len(matches))
matches[120]

In [ ]:
team_pairs = generate_team_pairs(matches)
list(team_pairs.items())[7:15]

In [ ]:
search_team_pairs('TEAM SPIRIT vs PSG LGD', matches)

## IoU

In [ ]:
df_youtube = pd.read_csv(YOUTUBE_DIR / 'XTngObUROMo.csv')
df_youtube.columns = ['video_id', 'cluster', 'clock_start', 'clock_end', 'start', 'end']
df_youtube['time'] = df_youtube['start'] 
df_youtube = TimeTable(df_youtube)
df_youtube

player_action_moments vs youtube

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
plot_player_signals(
    player,
    ax=ax,
    use_minutes=True,
    hp=True,
    action_moments=True,
    action_moments_line_level=0,
    df_youtube=df_youtube,
    youtube_line_level=-150,
)

plt.grid(True)
plt.show()

match_action_moments vs youtube

In [ ]:
fig, ax = plt.subplots(figsize=(15, 5))
plot_player_signals(
    player,
    ax=ax,
    use_minutes=True,
    df_youtube=df_youtube,
    youtube_line_level=-1,
    match_action_moments=True,
    match_action_moments_line_level=0,
)

ax.set_ylim(-10, 10)
plt.grid(True)
plt.show()

In [ ]:
youtube_moments = (
    df_youtube[['start', 'end']]
    .to_dict('records')
)
youtube_moments

In [ ]:
match_moments = match.action_moments[['start', 'end']].to_dict('records')

In [ ]:
calculate_iou(match_moments, youtube_moments)